##### 导入库函数

In [2]:
import pandas as pd
import plotly.graph_objs as go
import scipy.signal as signal
import numpy as np

##### 程序所需AES函数和汉明重量函数

In [ ]:
# AES仿真函数
s_box = {
    0 : ['0x63', '0x7c', '0x77', '0x7b', '0xf2', '0x6b', '0x6f', '0xc5', '0x30', '0x01', '0x67', '0x2b', '0xfe', '0xd7', '0xab', '0x76'],
    1 : ['0xca', '0x82', '0xc9', '0x7d', '0xfa', '0x59', '0x47', '0xf0', '0xad', '0xd4', '0xa2', '0xaf', '0x9c', '0xa4', '0x72', '0xc0'],
    2 : ['0xb7', '0xfd', '0x93', '0x26', '0x36', '0x3f', '0xf7', '0xcc', '0x34', '0xa5', '0xe5', '0xf1', '0x71', '0xd8', '0x31', '0x15'],
    3 : ['0x04', '0xc7', '0x23', '0xc3', '0x18', '0x96', '0x05', '0x9a', '0x07', '0x12', '0x80', '0xe2', '0xeb', '0x27', '0xb2', '0x75'],
    4 : ['0x09', '0x83', '0x2c', '0x1a', '0x1b', '0x6e', '0x5a', '0xa0', '0x52', '0x3b', '0xd6', '0xb3', '0x29', '0xe3', '0x2f', '0x84'],
    5 : ['0x53', '0xd1', '0x00', '0xed', '0x20', '0xfc', '0xb1', '0x5b', '0x6a', '0xcb', '0xbe', '0x39', '0x4a', '0x4c', '0x58', '0xcf'],
    6 : ['0xd0', '0xef', '0xaa', '0xfb', '0x43', '0x4d', '0x33', '0x85', '0x45', '0xf9', '0x02', '0x7f', '0x50', '0x3c', '0x9f', '0xa8'],
    7 : ['0x51', '0xa3', '0x40', '0x8f', '0x92', '0x9d', '0x38', '0xf5', '0xbc', '0xb6', '0xda', '0x21', '0x10', '0xff', '0xf3', '0xd2'],
    8 : ['0xcd', '0x0c', '0x13', '0xec', '0x5f', '0x97', '0x44', '0x17', '0xc4', '0xa7', '0x7e', '0x3d', '0x64', '0x5d', '0x19', '0x73'],
    9 : ['0x60', '0x81', '0x4f', '0xdc', '0x22', '0x2a', '0x90', '0x88', '0x46', '0xee', '0xb8', '0x14', '0xde', '0x5e', '0x0b', '0xdb'],
    10: ['0xe0', '0x32', '0x3a', '0x0a', '0x49', '0x06', '0x24', '0x5c', '0xc2', '0xd3', '0xac', '0x62', '0x91', '0x95', '0xe4', '0x79'],
    11: ['0xe7', '0xc8', '0x37', '0x6d', '0x8d', '0xd5', '0x4e', '0xa9', '0x6c', '0x56', '0xf4', '0xea', '0x65', '0x7a', '0xae', '0x08'],
    12: ['0xba', '0x78', '0x25', '0x2e', '0x1c', '0xa6', '0xb4', '0xc6', '0xe8', '0xdd', '0x74', '0x1f', '0x4b', '0xbd', '0x8b', '0x8a'],
    13: ['0x70', '0x3e', '0xb5', '0x66', '0x48', '0x03', '0xf6', '0x0e', '0x61', '0x35', '0x57', '0xb9', '0x86', '0xc1', '0x1d', '0x9e'],
    14: ['0xe1', '0xf8', '0x98', '0x11', '0x69', '0xd9', '0x8e', '0x94', '0x9b', '0x1e', '0x87', '0xe9', '0xce', '0x55', '0x28', '0xdf'],
    15: ['0x8c', '0xa1', '0x89', '0x0d', '0xbf', '0xe6', '0x42', '0x68', '0x41', '0x99', '0x2d', '0x0f', '0xb0', '0x54', '0xbb', '0x16']
}

def hextobin(word):  # 把十六进制转换成二进制
    word = bin(int(word, 16))[2:]
    for i in range(0, 8 - len(word)):  # 补全八位
        word = '0' + word
    return word


def bintohex(word):  # 把二进制转换十六进制
    word = hex(int(word, 2))
    if len(word) == 4:
        return word
    elif len(word) < 4:
        return word.replace('x', 'x0')  # 0x5-->0x05
     
def S_box(plaintext):  # s盒函数
    a = ''.join(s_box[int(plaintext[2], 16)][int(plaintext[3], 16)])
    return a

def xor_8(plaintext, key):  # 8位进行异或
    b = int(plaintext, 16)
    c = int(key, 16)
    xor_tmp = b ^ c
    a = format(xor_tmp, '#04x')
    return a

In [ ]:

# 计算汉明重量
def hamming_weight(x): # x为一个16进制字节
    x = bin(int(x, 16))[2:]
    hamming_weight = sum([int(bit) for bit in x])  # 统计二进制数中1的个数
    return hamming_weight

##### 读取波形数据

In [4]:
# 读取CSV文件，第二列为功耗波形数据
data = pd.read_csv('./trace/Trace000004.csv', header=None, usecols=[1])
data = data.squeeze()

In [ ]:
# 绘制图像
fig = go.Figure()
fig.add_trace(go.Scatter(x=data.index, y=data))
fig.show()

##### 仿真ASE中轮密钥加和字节代换操作

In [ ]:
# 对输入的一个16进制字节进行异或和s盒操作
def AES_xor_sbox(plaintext, key):
    return S_box(xor_8(plaintext, key))

##### 计算中间值汉明重量

In [ ]:
# 读取100个明文
df = pd.read_csv('Plaintext.csv', header=None)

key = ''
plaintext_list = []
for i in range(2, 400, 4):
    plaintext_list.append(str(df.iloc[i, 0]).replace(' ', ''))

print(plaintext_list)

In [ ]:
# 计算中间值汉明重量
def find_hamming_list(num):
    hamming_list = []
    # 第num个字节
    for i in range(0, 256): # 猜测密钥
        temp = []
        # 将i转换为16进制
        i_hex = hex(i)
        if len(i_hex) < 4:
            i_hex = i_hex.replace('x', 'x0')
        
        for j in range(100):
            temp.append(hamming_weight(AES_xor_sbox(plaintext_list[j][2*num:2*num+2], i_hex[2:4])))
        hamming_list.append(temp)
    #print(hamming_list)
    #print(len(hamming_list))
    return hamming_list

##### 切分

In [ ]:
# 滑动平均滤波
window_size = 50  # 设置滑动窗口大小
pad_width = (window_size + 1) // 2 # 边缘填充
window = np.ones(window_size) / float(window_size)
middle_smooth_data = np.convolve(data, window, 'valid')
# 边缘填充
middle_smooth_data = np.pad(middle_smooth_data, (pad_width, pad_width), mode='edge')

fig = go.Figure()
fig.add_trace(go.Scatter(x = list(range(len(middle_smooth_data))), y = middle_smooth_data)) # 滑动滤波图像
fig.show()

In [ ]:
# 查找峰值
neg_data = -middle_smooth_data
peaks , _ = signal.find_peaks(neg_data, distance = 450)
neg_peaks = -neg_data[peaks]

print(len(peaks))

sep_test = np.zeros(len(data))
for i in range(len(sep_test)):
    if i in peaks:
        sep_test[i] = -100

fig.add_trace(go.Scatter(x = list(range(len(sep_test))), y = sep_test)) # 绘制反向尖峰
fig.show()

##### 计算相关系数

In [ ]:
# 计算相关系数
def correlation(x, y):
    n = len(x)
    mean_x = sum(x)/n
    mean_y = sum(y)/n
    cov_xy = sum([(x[i]-mean_x)*(y[i]-mean_y) for i in range(n)])
    var_x = sum([(x[i]-mean_x)**2 for i in range(n)])
    var_y = sum([(y[i]-mean_y)**2 for i in range(n)])
    corr_coef = cov_xy / (var_x * var_y)**0.5
    return corr_coef

In [ ]:
# 遍历寻找相关系数大于0.6的密钥
def find_key_num(lst):
    for i in range(len(lst)):
        for j in range(len(lst[i])):
            if lst[i][j] > 0.6:
                key.join(hex(i)[2:].upper())
                print(hex(i)[2:].upper())
                break

In [ ]:
def find_key(num): # 0<=num<16
    trace_list = []
    correlation_list = []
    hamming_list = find_hamming_list(num)
    
    if num < 15:
        begin = peaks[num]
        end = peaks[num+1]
    elif num == 15:
        begin = peaks[num]
        end = sep_test[-1]
        
    # 取第num个字节对应的波形
    for i in range(1, 101):
        i_000 = str(i).zfill(3)
        data = pd.read_csv('Trace000{}.csv'.format(i_000), header=None, usecols=[1])
        data = data.squeeze()
        trace_list.append(data[begin:end])

    # 转置
    transposed_trace_list = list(map(list, zip(*trace_list)))

    # 计算相关系数
    for i in range(len(hamming_list)):
        temp = []
        for j in range(len(transposed_trace_list)):
            temp.append(correlation(hamming_list[i], transposed_trace_list[j]))
        correlation_list.append(temp)
    # print(len(correlation_list))
    
    if num == 0:
        fig = go.Figure()
        for i in range(len(correlation_list)):
            # 绘制图像
            fig.add_trace(go.Scatter(x = list(range(len(correlation_list[i]))), y = correlation_list[i])) 
        fig.show()

    find_key_num(correlation_list)

In [ ]:
for i in range(15):
    find_key(i)
print('key = '.format(key))